In [20]:
import os

In [21]:
%pwd

'f:\\End to End Machine learning project\\End-to-End-ML-Project'

In [22]:
os.chdir("f:\\End to End Machine learning project\\End-to-End-ML-Project") 

In [23]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir:  Path
    all_schema: dict

In [24]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories
from ml_project.entity.config_entity import DataIngestionConfig

In [25]:
class configurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_roots])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    def get_data_validation_config(self)-> DataValidationConfig:
        config = self.config.data_validation
        schema =  self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(root_dir= config.root_dir,
                                                      STATUS_FILE = config.STATUS_FILE,
                                                      unzip_data_dir = config.unzip_data_dir,
                                                      all_schema = schema)
        return data_validation_config

In [26]:
# update the components
import os
from ml_project.logging import logger
import pandas as pd

In [27]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
    
    #making function validate all columns
    def validate_all_columns(self)->bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation staus: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation staus: {validation_status}")
            return validation_status
        except Exception as e:
            raise e

In [28]:
try:
    config = configurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config= data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-03-26 01:30:22,336: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-26 01:30:22,344: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-26 01:30:22,352: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-26 01:30:22,355: INFO: common: created directory at: artifacts]
[2024-03-26 01:30:22,356: INFO: common: created directory at: artifacts/data_validation]
